# Severstal Steel Defect EDA

In [ ]:
IMAGE_DIR = './data/severstal-steel-defect-detection/train_images/'
ANNOTATIONS_FILE = './data/severstal-steel-defect-detection/train.csv'
# IMAGE_DIR = './data/severstal-steel-defect-detection/test_images/'
# ANNOTATIONS_FILE = '/home/ryan/Downloads/submission (8).csv'
# IMAGE_DIR = './data/severstal-steel-defect-detection/test_images/'
# ANNOTATIONS_FILE = 'submission.csv'

In [ ]:
from collections import defaultdict

import numpy as np

from steel_seg.utils import rle_to_dense
from steel_seg.dataset.dataset_utils import load_annotations


In [ ]:
anns = load_annotations(ANNOTATIONS_FILE)

In [ ]:
len(anns)

In [ ]:
anns[list(anns.keys())[1]]

## Visualize Examples

In [ ]:
import os

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

def visualize_segmentations(img, anns):
    vis_img = img.copy()
    
    colours = [[0, 235, 235], [0, 210, 0], [0, 0, 255], [255, 0, 255]]
    for i in range(4):
        mask = anns[:, :, i]
        if np.any(mask):
            print(f'Class {i}')
        kernel = np.ones((10, 10), np.uint8) 
        dilated_mask = cv2.dilate(mask, kernel, iterations=1)
        contour_mask = dilated_mask - mask
        for c in range(3):
            vis_img[contour_mask == 1, c] = colours[i][c]
    return vis_img

def visualize_img_with_masks(img_path, ann_dict):
    num_classes = 4
    img = np.array(Image.open(img_path))

    masks = []
    for cls_id in range(1, num_classes + 1):
        masks.append(rle_to_dense(ann_dict[str(cls_id)], img_height=img.shape[0], img_width=img.shape[1]))
        
    mask = np.stack(masks, axis=-1)
    mask.astype(np.uint8)
    vis_img = visualize_segmentations(img, mask)
    plt.figure(figsize=(10, 3))
    plt.imshow(vis_img)
    plt.show()

def visualize_imgs(anns):
    count = 120

    for img_id, ann in anns.items():   
        #if ann['1'] != '' or ann['2'] != '' or ann['3'] != '' or ann['4']:
        print(f'**********{img_id}**********')
        visualize_img_with_masks(os.path.join(IMAGE_DIR, img_id), ann)
        count -= 1
        if count <= 0:
            return


visualize_imgs(anns)


## Training Set Class Breakdown

In [ ]:
counts = {'no_labels': 0,
          '1_only': 0,
          '2_only': 0,
          '3_only': 0,
          '4_only': 0,
          'multiple_labels_2': 0,
          'multiple_labels_3': 0,
          'multiple_labels_4': 0}

for ann in anns.values():
    labels = []
    for k, v in ann.items():
        if v != '':
            labels.append(k)

    if len(labels) == 0:
        counts['no_labels'] += 1
    elif len(labels) == 1:
        counts[labels[0] + '_only'] += 1
    elif len(labels) >= 1:
        counts['multiple_labels_' + str(len(labels))] += 1

for k, v in counts.items():
    print(f'{k}: {v}')

In [ ]:
no_labels: 1184
1_only: 65
2_only: 6
3_only: 447
4_only: 79
multiple_labels_2: 20
multiple_labels_3: 0
multiple_labels_4: 0

In [ ]:
no_labels: 931
1_only: 315
2_only: 26
3_only: 394
4_only: 84
multiple_labels_2: 50
multiple_labels_3: 1
multiple_labels_4: 0

In [ ]:
From probing public test set:
no_labels: 6172
1_only: 128
2_only: 43
3_only: 741
4_only: 120
 

